<a href="https://colab.research.google.com/github/gh-ita/SafetyScan/blob/main/image_similarity_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-gpu-cu11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 37.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import torch
from torchvision import transforms
import numpy as np
import os
import matplotlib.pyplot as plt
import faiss
from sklearn.decomposition import PCA

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = SentenceTransformer('clip-ViT-B-32')

In [ ]:
model = model.to(device)

In [ ]:
train_img = "/content/drive/MyDrive/Construction-Site-Safety-5/train/images"
train_img_list = os.listdir(train_img)

In [ ]:
img_embeddings = []
for image_path in train_img_list:
    if image_path != '.ipynb_checkpoints':
        img = Image.open(os.path.join(train_img, image_path))
        image_embedding = model.encode(img)
        img_embeddings.append(image_embedding)

In [ ]:
init_length = len(img_embeddings[0])
same_length = 1
for emb in img_embeddings:
    if len(emb) != init_length:
        same_length = 0
print(same_length)

In [ ]:
len(img_embeddings)

# **Clustering**

First iteration : 20 centroids

In [ ]:
niter = 20
verbose = True
img_array = np.array(img_embeddings)
d = img_array.shape[1]
obj_vals = {}

In [ ]:
f_iter_ncentroids = 7
f_iter_kmeans = faiss.Kmeans(d, f_iter_ncentroids, niter=niter, verbose=verbose)
f_obj_val = f_iter_kmeans.train(img_embeddings)
obj_vals[f_iter_ncentroids] = f_obj_val

In [ ]:
f_iter_kmeans.obj

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(f_iter_kmeans.obj)
plt.show()

Second iteration : 14

In [ ]:
s_iter_ncentroids = 14
s_iter_kmeans = faiss.Kmeans(d, s_iter_ncentroids, niter=niter, verbose=verbose)
s_obj_val = s_iter_kmeans.train(img_embeddings)
obj_vals[s_iter_ncentroids] = s_obj_val

In [ ]:
s_iter_kmeans.obj

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(s_iter_kmeans.obj)
plt.show()

Third iteration: 28

In [ ]:
t_iter_ncentroids = 28
t_iter_kmeans = faiss.Kmeans(d, t_iter_ncentroids, niter=niter, verbose=verbose)
t_obj_val = t_iter_kmeans.train(img_embeddings)
obj_vals[t_iter_ncentroids] = t_obj_val

In [ ]:
t_iter_kmeans.obj

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(t_iter_kmeans.obj)
plt.show()

Fourth iteration : 56

In [ ]:
fo_iter_ncentroids = 56
fo_iter_kmeans = faiss.Kmeans(d, fo_iter_ncentroids, niter=niter, verbose=verbose)
fo_obj_val = fo_iter_kmeans.train(img_embeddings)
obj_vals[fo_iter_ncentroids] = fo_obj_val

In [ ]:
fo_iter_kmeans.obj

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(fo_iter_kmeans.obj)
plt.show()

Fifth iteration : 70

In [ ]:
fi_iter_ncentroids = 70
fi_iter_kmeans = faiss.Kmeans(d, fi_iter_ncentroids, niter=niter, verbose=verbose)
fi_obj_val = fi_iter_kmeans.train(img_embeddings)
obj_vals[fi_iter_ncentroids] = fi_obj_val

In [ ]:
fi_iter_kmeans.obj

Sixth iteration : 84

In [ ]:
si_iter_ncentroids = 84
si_iter_kmeans = faiss.Kmeans(d, si_iter_ncentroids, niter=niter, verbose=verbose)
si_obj_val = si_iter_kmeans.train(img_embeddings)
obj_vals[si_iter_ncentroids] = si_obj_val

In [ ]:
si_iter_kmeans.obj

Seventh iteration : 98

In [ ]:
se_iter_ncentroids = 98
se_iter_kmeans = faiss.Kmeans(d, se_iter_ncentroids, niter=niter, verbose=verbose)
se_obj_val = se_iter_kmeans.train(img_embeddings)
obj_vals[se_iter_ncentroids] = se_obj_val

In [ ]:
se_iter_kmeans.obj

Eighth iteration : 112

In [ ]:
ei_iter_ncentroids = 112
ei_iter_kmeans = faiss.Kmeans(d, ei_iter_ncentroids, niter=niter, verbose=verbose)
ei_obj_val = ei_iter_kmeans.train(img_embeddings)
obj_vals[ei_iter_ncentroids] = ei_obj_val

In [ ]:
ei_iter_kmeans.obj

Ninth iteration : 126

In [ ]:
ni_iter_ncentroids = 126
ni_iter_kmeans = faiss.Kmeans(d, ni_iter_ncentroids, niter=niter, verbose=verbose)
ni_obj_val = ni_iter_kmeans.train(img_embeddings)
obj_vals[ni_iter_ncentroids] = ni_obj_val

In [ ]:
ni_iter_kmeans.obj

Thenth iteration : 134

In [ ]:
te_iter_ncentroids = 134
te_iter_kmeans = faiss.Kmeans(d, te_iter_ncentroids, niter=niter, verbose=verbose)
te_obj_val = te_iter_kmeans.train(img_embeddings)
obj_vals[te_iter_ncentroids] = te_obj_val

Eleventh iteration: 162

In [ ]:
el_iter_ncentroids = 162
el_iter_kmeans = faiss.Kmeans(d, el_iter_ncentroids, niter=niter, verbose=verbose)
el_obj_val = el_iter_kmeans.train(img_embeddings)
obj_vals[el_iter_ncentroids] = el_obj_val

Twelfth iteration : 190

In [ ]:
tw_iter_ncentroids = 190
tw_iter_kmeans = faiss.Kmeans(d, tw_iter_ncentroids, niter=niter, verbose=verbose)
tw_obj_val = tw_iter_kmeans.train(img_embeddings)
obj_vals[tw_iter_ncentroids] = tw_obj_val

In [ ]:
print(obj_vals)

### Objective function value per number of centroids

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(obj_vals.keys(), obj_vals.values())
plt.show()